In [3]:
import cv2
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.logger import setup_logger
import os
import json

# 配置 Detectron2
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))  # 使用预训练模型
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # 预训练模型权重
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # 设置预测阈值
cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 使用GPU，如果可用

# 创建预测器
predictor = DefaultPredictor(cfg)

# 读取COCO格式的图像ID映射
with open('output/Ground Truth.json', 'r') as f:
    coco_data = json.load(f)
image_id_map = {image['file_name']: image['id'] for image in coco_data['images']}
print(image_id_map)

# 存储COCO格式的预测结果
coco_results = []

annotation_id = 0  # 用于唯一标识每个预测框

# 遍历images文件夹中的所有图像文件
image_folder = 'images'
for image_filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_filename)
    image = cv2.imread(image_path)
    print(f"Processing {image_filename}...")
    image_id = image_id_map.get(image_filename)

    if image is None:
        continue

    # 进行目标检测
    outputs = predictor(image)

    # 获取检测框、得分、标签
    instances = outputs["instances"]
    boxes = instances.pred_boxes.tensor.cpu().numpy()  # 将张量移到CPU并转换为NumPy数组
    scores = instances.scores.cpu().numpy()
    labels = instances.pred_classes.cpu().numpy()

    # 过滤出仅包含行人的检测结果（COCO中，行人类别ID为0）
    person_indices = labels == 0  # COCO中，行人类别ID为0
    boxes = boxes[person_indices]
    scores = scores[person_indices]
    labels = labels[person_indices]

    # 保存检测结果可视化图像
    save_path = 'detectResult/Fast R-CNN'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(f"{save_path}/{image_filename}", out.get_image()[:, :, ::-1])
    

    # 存储检测结果到COCO格式
    for box, score in zip(boxes, scores):
        bbox = [int(box[0]), int(box[1]), int(box[2] - box[0]), int(box[3] - box[1])]
        coco_results.append({
            "image_id": image_id,
            "category_id": 0,
            "bbox": bbox,
            "score": float(score)
        })
        annotation_id += 1  # 增加标注ID

# 将结果保存为COCO格式的JSON文件
output_filename = 'output/Fast R-CNN.json'
with open(output_filename, 'w') as f:
    json.dump(coco_results, f)
print(f"Detection results saved to {output_filename}")


{'b71184b9-1.jpg': 0, 'b2dd240c-2.jpg': 1, 'e628b968-3.jpg': 2, 'c1c0fe3c-4.jpg': 3, '16bf9a67-0.jpg': 4, 'ad3956f8-9.jpg': 5, '0dd20243-8.jpg': 6, 'd9a9c2fa-5.jpg': 7, '9a07eb57-6.jpg': 8, '796ec187-7.jpg': 9}
Processing 0dd20243-8.jpg...
Processing 16bf9a67-0.jpg...
Processing 796ec187-7.jpg...
Processing 9a07eb57-6.jpg...
Processing ad3956f8-9.jpg...
Processing b2dd240c-2.jpg...
Processing b71184b9-1.jpg...
Processing c1c0fe3c-4.jpg...
Processing d9a9c2fa-5.jpg...
Processing e628b968-3.jpg...
Detection results saved to output/Fast R-CNN.json
